In [ ]:
#|default_exp dove_classifier_app

In [ ]:
#This notebook is my workbook for the 2nd chapter in the FastAI course.
# https://github.com/fastai/fastbook/blob/master/02_production.ipynb
!pip install -Uqq fastai
!pip install -Uqq gradio


In [ ]:
from fastai.vision.widgets import *


In [ ]:
#|export
from fastai.vision.all import *
import gradio as gr

In [ ]:
# My goal is to have train a Dove classifier predict the kind of dove given an input image of a dove.
# The dataset used here is a tiny subset of the awesome collection of bird pictures 
# at https://www.kaggle.com/datasets/gpiosenka/100-bird-species 
# This Dove classifier model is trained only on 3 species of doves: zebra doves, rock doves and mourning doves
bird_types = 'ZEBRA DOVE','ROCK DOVE','MOURNING DOVE'
#print(training_data_path)
#fns = get_image_files(path + '/train/')
#fns
dove_img_train = get_image_files("/kaggle/input/doves-train-dataset/Doves/train/")
dove_img_train
corrupt_images = verify_images(dove_img_train)
corrupt_images

In [ ]:
# The dataset has separate folders for training, validation and tests.
# I've used RandomSplitter to match the FastAI course. 
# So the images in the training and validation folders have been clubbed together.
doves = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
# Just figured out that a better alternative would've been to use GrandparentSplitter
# https://github.com/fastai/fastai/blob/96f0e2027c8fd82eef40984faf14c1c81e0eb031/fastai/data/transforms.py#L131
#??RandomSplitter
#??GrandparentSplitter
doves

In [ ]:
#doves
path = Path("/kaggle/input/doves-train-dataset/Doves/train/")
dls = doves.dataloaders(path)
dls

In [ ]:
dls.train.show_batch(max_n=4, nrows=1)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:

# Decided to try the Resnet50 model and trained it for 6 epochs.
# With 3 epochs, the model fumbled with an image of a Rock Dove in the test set
# and incorrectly classified it as a Mourning Dove. 
#This was resolved with the additional training over the course of the 6 epochs
learn = vision_learner(dls, resnet50, metrics=error_rate)
learn.fine_tune(6)

#However the validation loss and error rates seem to degrade with the additional training 

In [ ]:

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:

interp.plot_top_losses(5, ncols=1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner
#??ImageClassifierCleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()

In [ ]:
#The pickledT model file is exported as dove_classifier.pkl
learn.export('dove_classifier.pkl')


In [ ]:

path = Path('/kaggle/working/')
path.ls(file_exts='.pkl')
#/kaggle/working/ #/kaggle/input/doves-train-dataset
learn_inf = load_learner('/kaggle/working/dove_classifier.pkl')

In [ ]:
#path
learn_inf

In [ ]:
learn_inf.predict('/kaggle/input/doves-dataset/Birds/test/MOURNING DOVE/5.jpg')

In [ ]:
learn_inf.predict('/kaggle/input/doves-dataset/Birds/test/ROCK DOVE/3.jpg')


In [ ]:
learn_inf.predict('/kaggle/input/doves-dataset/Birds/test/ZEBRA DOVE/5.jpg')

In [ ]:
#Click handler to display the uploaded image in a widget

out_pl   = widgets.Output()
lbl_pred = widgets.Label()
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
   
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = "Prediction: "+ pred + "\nProbability: "+ str(probs[pred_idx])



In [ ]:
btn_upload = widgets.FileUpload()
#btn_upload

In [ ]:
#Create a button widget to upload an image for prediction


#Create a button widget to invoke the model
btn_run = widgets.Button(description='Classify')
btn_run.on_click(on_click_classify)
#hide_output
VBox([widgets.Label('Select your dove!'), 
      btn_upload, btn_run, out_pl, lbl_pred])

In [ ]:
#The following steps are to create a web app to upload a dove picture and to use the Dove classifer to classify it

In [ ]:
#|export
# Following the convention used in 
# https://huggingface.co/spaces/jph00/testing/blob/main/app.ipynb
dove_categories = ('MOURNING DOVE', 'ROCK DOVE', 'ZEBRA DOVE')
def classify_image(img):
    pred,idx,probs = learn_inf.predict(img)
    return dict(zip(dove_categories, map(float,probs)))

In [ ]:
#|export
image = gr.Image(width=192,height=192)
label = gr.Label()
#examples = ['dog.jpg', 'cat.jpg', 'dunno.jpg']

#intf = gr.Interface(fn=classify_image, inputs=image, outputs=label, examples=examples)
intf = gr.Interface(fn=classify_image, inputs=image, outputs=label)
intf.launch(inline=False)

In [ ]:
#from nbdev.export import notebook2script


In [22]:
import nbdev
??nbdev.export.nb_export(nbname='dove_classifier_app.ipynb',lib_path='/kaggle/working/',debug=True)
#nbdev.export.nb_export('dove_clasifier_app.ipynb', '.')
#print('Export successful')

Signature:
nbdev.export.nb_export(
    nbname,
    lib_path=None,
    procs=None,
    debug=False,
    mod_maker=<class 'nbdev.maker.ModuleMaker'>,
    name=None,
)
Source:   
def nb_export(nbname, lib_path=None, procs=None, debug=False, mod_maker=ModuleMaker, name=None):
    "Create module(s) from notebook"
    if lib_path is None: lib_path = get_config().lib_path
    exp = ExportModuleProc()
    nb = NBProcessor(nbname, [exp]+L(procs), debug=debug)
    nb.process()
    for mod,cells in exp.modules.items():
        all_cells = exp.in_all[mod]
        nm = ifnone(name, getattr(exp, 'default_exp', None) if mod=='#' else mod)
        if not nm:
            warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
                 "Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.\n"
                 "See https://nbdev.fast.ai/getting_started.html for more information.")
            return
        mm = mod_maker(dest=lib_path, name=nm, nb_path

In [20]:
nbdev_migrate

NameError: name 'nbdev_migrate' is not defined

In [ ]:
#noteboook2script(dove_classifier_app.ipynb)

In [ ]:
#!pip install voila
#!jupyter serverextension enable --sys-prefix voila